In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizerFast, BitsAndBytesConfig, MistralForCausalLM
import json

In [2]:
torch.set_default_device('cuda')

In [3]:
# Initialize the tokenizer for "alexsherstinsky/Mistral-7B-v0.1-sharded"
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", device_map={'':torch.cuda.current_device()})

In [4]:
with open('../data_finetuning/MERGED_DETECTIVE.txt', 'r') as file:
    content = file.read()

In [5]:
len(content)

5405190

In [6]:
instruction_text = "These are the first lines of a literary work. Continue the text below, closely following its style and content."

In [ ]:
tokens = tokenizer(content, truncation=False, return_tensors="pt", add_special_tokens=False)

In [ ]:
new_dataset = {'text': []}

current_chunk_size = 150
next_chunk_size = 250

for i in range(0, tokens.input_ids.size(1) - next_chunk_size, current_chunk_size):
    end_index_current_chunk = min(i + current_chunk_size, tokens.input_ids.size(1))
    end_index_next_chunk = min(end_index_current_chunk + next_chunk_size, tokens.input_ids.size(1))

    current_chunk_token_ids = tokens.input_ids[0, i:end_index_current_chunk]
    next_chunk_token_ids = tokens.input_ids[0, end_index_current_chunk:end_index_next_chunk]

    current_chunk_text = tokenizer.decode(current_chunk_token_ids, skip_special_tokens=True)
    next_chunk_text = tokenizer.decode(next_chunk_token_ids, skip_special_tokens=True)

    new_chat = f"<s>[INST]{instruction_text} {current_chunk_text} [/INST] {next_chunk_text}</s>"
    new_dataset['text'].append(new_chat)  # Add to the 'text' list within the dictionary


In [ ]:
with open('AUTRE/data/finetuning_instructions_dataset_POLICIER.json', 'w', encoding='utf-8') as f:
    json.dump(new_dataset, f, ensure_ascii=False, indent=4)

In [ ]:
if new_dataset['text']:
    print(new_dataset['text'][1])
else:
    print("The new_dataset['text'] list is empty.")

In [11]:
if new_dataset['text']:
    print(new_dataset['text'][0])
else:
    print("The new_dataset['text'] list is empty.")

<s>[INST]These are the first lines of a literary work. Continue the text below, closely following its style and content.  EH ! de quoi diable se mêle mademoiselle Ursule, ma très-honorée soeur ? Comment ! à mon âge je ne serai pas maître de mes actions ? pense-t-elle que le capitaine Constantin de Cassard, qui, de son bord, a fait trembler plus d’une fois les Anglais, se laissera mener par une femme ? Corbleu ! n’a-t-ii pas assez de chagrin de savoir que l’armée navale va compter de nouveaux succès, que la palme de la victoire sera cueillie [/INST] par les Français sans qu’il puisse y mettre le grapin ? Nom de tribord et de babord ! sans ce maudit biscayen qui m’a frappé la jambe et que je brûle de rendre à l’ennemi, je ne resterais pas une heure dans les domaines des comtes de Cassard, mes nobles aïeux, et je préférerais entendre le bruit du canon et celui des brûlots de l’ennemi, plutôt que votre éternel bavardage.  Calmez-vous, capitaine, je vous en prie, lui répond mademoiselle Urs